In [31]:
import imaplib
import email
from email.header import decode_header
from textblob import TextBlob

def decode_subject(encoded_subject):
    decoded, encoding = decode_header(encoded_subject)[0]
    if isinstance(decoded, bytes):
        decoded = decoded.decode(encoding or 'utf-8')
    return decoded

def decode_body(body):
    if isinstance(body, bytes):
        try:
            # Try decoding using UTF-8
            return body.decode('utf-8')
        except UnicodeDecodeError:
            # If decoding fails, try ignoring errors or replace with a placeholder character
            return body.decode('utf-8', errors='ignore')  # or errors='replace'
    elif isinstance(body, str):
        return body  # If the body is already a string, no decoding is needed
    else:
        return str(body)  # Convert to string if neither bytes nor string

def fetch_emails(server, username, password, folder="inbox"):
    mail = imaplib.IMAP4_SSL(server)
    mail.login(username, password)
    mail.select(folder)

    status, messages = mail.search(None, "ALL")
    email_ids = messages[0].split()

    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, "(RFC822)")
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                email_message = email.message_from_bytes(response_part[1])
                subject = decode_subject(email_message["Subject"])
                sender = email.utils.parseaddr(email_message["From"])[1]
                body = ""

                if email_message.is_multipart():
                    for part in email_message.walk():
                        if part.get_content_type() == "text/plain":
                            body = part.get_payload(decode=True)
                else:
                    body = email_message.get_payload(decode=True)

                # print(f"Subject: {subject}")
                # print(type(decode_body(subject)))
                # print(f"From: {sender}")
                # print(f"Body:\n{decode_body(body)}")
                print(analyze_text_with_textblob(subject+decode_body(body))*5, '/ 5 is the Positivity of the mail sent by', sender)
                if analyze_text_with_textblob(subject+decode_body(body))*5 > 2.5:
                    print('and it is a Positive response.')
                else:
                    print('and it is a Negative response.')
    mail.logout()


In [ ]:
# Replace these with your email server details
email_server = "imap.gmail.com"
email_username = "*********"
email_password = "*********"

In [32]:
def analyze_text_with_textblob(paragraph):
    blob = TextBlob(paragraph)
    # Sentiment analysis
    positive_score = blob.sentiment.polarity
    return positive_score

In [33]:
#Fetch emails
fetch_emails(email_server, email_username, email_password)

3.1666666666666665 / 5 is the Positivity of the mail sent by account-update@amazon.in
and it is a Positive response.
1.163265306122449 / 5 is the Positivity of the mail sent by auto-confirm@amazon.in
and it is a Negative response.
0.5319264069264069 / 5 is the Positivity of the mail sent by no-reply@accounts.google.com
and it is a Negative response.
0.9652857204348687 / 5 is the Positivity of the mail sent by store-news@amazon.in
and it is a Negative response.
0.8333333333333333 / 5 is the Positivity of the mail sent by ship-confirm@amazon.in
and it is a Negative response.
2.5 / 5 is the Positivity of the mail sent by store-news@amazon.in
and it is a Negative response.
0.048611111111111154 / 5 is the Positivity of the mail sent by store-news@amazon.in
and it is a Negative response.
1.25 / 5 is the Positivity of the mail sent by store-news@amazon.in
and it is a Negative response.
1.0416666666666667 / 5 is the Positivity of the mail sent by order-update@amazon.in
and it is a Negative res

KeyboardInterrupt: 